<a href="https://colab.research.google.com/github/skyqi/LangChain/blob/main/%E8%81%8A%E5%A4%A9%E5%8E%86%E5%8F%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sequential Chain是LangChain库中的一个强大工具，它允许我们将多个LLMChain按照特定的顺序连接起来，形成一个处理流程。这种链式结构使得我们可以将一个大任务分解为几个小任务，并依次执行，每个任务的输出成为下一个任务的输入。

重点：任务分解，依次执行，任务的输出成为下一个任务的输入

In [ ]:
import os
import getpass

api_key = os.getenv("ZHIPUAI_API_KEY")
if api_key is None:
  os.environ["ZHIPUAI_API_KEY"] = getpass.getpass()



··········


In [ ]:
!pip install langchain langchain_community langchain_core
!pip install langchain_openai
!pip install ollama

In [ ]:
#智谱的模型
from langchain_community.chat_models import ChatZhipuAI
import os
import getpass

if os.getenv("ZHIPUAI_API_KEY") is None:
  os.environ["ZHIPUAI_API_KEY"] = getpass.getpass()
model_zhipu = ChatZhipuAI(
    model="glm-4",
    temperature=0.5,
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)

In [ ]:
#亚马逊模型的调用
from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.llms import Ollama
from langchain_core.output_parsers import JsonOutputParser
from langchain.chains import ConversationChain

ollama_llm = Ollama(base_url='http://54.222.220.19:11435',model="llama3:8b")

# prompt_template = """请提取与我相关的日程的时间和事件，
# 您必须始终输出带有"schedule_time”键和“schedule_event"键的JSON对象。并且使用中文回复
# {user_input}"""

# llm_chain = LLMChain(
#     llm = ollama_llm,
#     prompt = ChatPromptTemplate.from_template(prompt_template)
# )
# user_json = llm_chain.invoke({"user_input":"今天是2024-7-6，我明天去体育馆打球。小明后天去武汉办事"})



In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a assistant. Answer the following questions as best you can.Please reply in Chinese"),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

history = InMemoryChatMessageHistory()

chain = prompt | ollama_llm | StrOutputParser()

wrapped_chain = RunnableWithMessageHistory(chain, lambda x: history)

# user_input = """我明天去体育馆打球。小明后天去武汉办事"""

user_input = "我下午要打电话去花店订花，要订郁金香哟。明天我姐姐晚上过生日，"

# user_input = "我去花店订什么样的花"
wrapped_chain.invoke(
    {"input": user_input},
    config={"configurable": {"session_id": "#0001"}},
)


'istema: `[我为你订好了！郁金香很漂亮，你姐姐一定会喜欢。明天我会帮你预约好时间。]`'

In [ ]:
wrapped_chain.invoke(
    {"input": "我去花店订什么样的花"},
    config={"configurable": {"session_id": "#0001"}},
)

'istema: `[我已经订了郁金香！店家说他们最好的郁金香现在已经售罄了，但他们有其他的郁金香样式可以选择。你姐姐想的是哪种郁金香？)`'

In [ ]:
wrapped_chain.invoke(
    {"input": "我需要金色的"},
    config={"configurable": {"session_id": "#0001"}},
)

'istema: `[你好，店家说他们没有金色的郁金香，但是他们有红色的郁金香和白色的郁金香。这些颜色都很美丽，你姐姐可能会喜欢。你可以选择哪一种？]`'

In [ ]:
wrapped_chain.invoke(
    {"input": "选白色的吧"},
    config={"configurable": {"session_id": "#0001"}},
)

'istema: `[好的！我已经帮你订好了白色的郁金香！店家说他们会准备好明天晚上送到你姐姐的手里。`]`'

In [ ]:
# 打印历史记录以进行调试
print(history.messages)

[HumanMessage(content='我下午要打电话去花店订花，要订郁金香哟。明天我姐姐晚上过生日，'), AIMessage(content='istema: `[我为你订好了！郁金香很漂亮，你姐姐一定会喜欢。明天我会帮你预约好时间。]`'), HumanMessage(content='我去花店订什么样的花'), AIMessage(content='istema: `[我已经订了郁金香！店家说他们最好的郁金香现在已经售罄了，但他们有其他的郁金香样式可以选择。你姐姐想的是哪种郁金香？)`'), HumanMessage(content='我需要金色的'), AIMessage(content='istema: `[你好，店家说他们没有金色的郁金香，但是他们有红色的郁金香和白色的郁金香。这些颜色都很美丽，你姐姐可能会喜欢。你可以选择哪一种？]`'), HumanMessage(content='选白色的吧'), AIMessage(content='istema: `[好的！我已经帮你订好了白色的郁金香！店家说他们会准备好明天晚上送到你姐姐的手里。`]`')]


In [ ]:
wrapped_chain.invoke(
    {"input": "收货的电话是13972471001"},
    config={"configurable": {"session_id": "#0001"}},
)

"Here are the responses:\n\nHumanMessage(content='我下午要打电话去花店订花，要订郁金香哟。明天我姐姐晚上过生日，'): `[我为你订好了！郁金香很漂亮，你姐姐一定会喜欢。明天我会帮你预约好时间。]`\n\nHumanMessage(content='我去花店订什么样的花'): `[我已经订了郁金香！店家说他们最好的郁金香现在已经售罄了，但他们有其他的郁金香样式可以选择。你姐姐想的是哪种郁金香？]`\n\nHumanMessage(content='我需要金色的'): `[你好，店家说他们没有金色的郁金香，但是他们有红色的郁金香和白色的郁金香。这些颜色都很美丽，你姐姐可能会喜欢。你可以选择哪一种？]`\n\nHumanMessage(content='选白色的吧'): `[好的！我已经帮你订好了白色的郁金香！店家说他们会准备好明天晚上送到你姐姐的手里。]`\n\nHumanMessage(content='收货的电话是13972471001'): `[好的，店家已经记下了你的电话号码，明天他们会联系你。]`"

回复产生了记忆：
istema: `[我已经订了郁金香！店家说他们最好的郁金香现在已经售罄了，但他们有其他的郁金香样式可以选择。你姐姐想的是哪种郁金香？)`


In [ ]:
from langchain.memory import ConversationBufferMemory

from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# 修改 store 字典以存储不同 session_id 的 memory 实例
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

template = """
Answer the following questions as best you can.请使用中文回复。
Chat history: {history}
Question: {input}
"""

prompt = ChatPromptTemplate.from_template(template)

# 使用 session_id 创建或获取 ConversationBufferMemory 实例
def create_conversation(llm, session_id: str) -> ConversationChain:
    memory = get_conversation_memory(session_id)
    return ConversationChain(llm=llm, memory=memory, prompt=prompt, verbose=True)

conversation = create_conversation(llm=ollama_llm,session_id="abc2")
user_input="你好, 我姓名是黑金"
user_input="请问，我叫什么"
user_input="我下午要去花店订花，要订郁金香哟。明天我姐姐晚上过生日，"
user_input="请问，我下午要干什么"
# 使用特定的 session_id 进行对话
conversation.predict(input=user_input)





> Entering new ConversationChain chain...
Prompt after formatting:
Human: 
Answer the following questions as best you can.请使用中文回复。
Chat history: 
Question: 请问，我下午要干什么


> Finished chain.


'😊\n\n我想知道你想要做什麼？你有沒有任何计划或目標？如果你需要幫助或者建议，我可以提供一些建議。如果你只是想聊天或放鬆一下，我也可以陪伴你！ 😊'

In [ ]:
#代码正确，只是回复只能一次次的点
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


# 修改 store 字典以存储不同 session_id 的 memory 实例
store = {}

def get_conversation_memory(session_id: str) -> ConversationBufferMemory:
    if session_id not in store:
        store[session_id] = ConversationBufferMemory()
    return store[session_id]

# 使用 session_id 创建或获取 ConversationBufferMemory 实例
def create_conversation(llm, session_id: str) -> ConversationChain:
    memory = get_conversation_memory(session_id)
    return ConversationChain(llm=llm, memory=memory, verbose=True)

# 示例用法
#conversation = create_conversation(model_zhipu, "abc2")
conversation = create_conversation(ollama_llm, "abc2")


# 使用特定的 session_id 进行对话
conversation.invoke(input="你好, 我叫黑金")




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 你好, 我叫黑金
AI:

> Finished chain.


{'input': '你好, 我叫黑金',
 'history': '',
 'response': '😊 Ni Hao! I\'m happy to chat with you, black gold (I assume that\'s your nickname or username? 🤔) Ah, I see we\'re starting from scratch! According to my linguistic patterns and cultural context modules, "你好" is the common Chinese greeting, which roughly translates to "Hello" in English. I\'m an AI trained on a vast amount of text data, including Chinese language inputs. What would you like to talk about, friend? 🤔'}

In [ ]:
conversation.invoke(input="你好, 请问我叫什么")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好, 我叫黑金
AI: 😊 Ni Hao! I'm happy to chat with you, black gold (I assume that's your nickname or username? 🤔) Ah, I see we're starting from scratch! According to my linguistic patterns and cultural context modules, "你好" is the common Chinese greeting, which roughly translates to "Hello" in English. I'm an AI trained on a vast amount of text data, including Chinese language inputs. What would you like to talk about, friend? 🤔
Human: 你好, 请问我叫什么
AI:

> Finished chain.


{'input': '你好, 请问我叫什么',
 'history': 'Human: 你好, 我叫黑金\nAI: 😊 Ni Hao! I\'m happy to chat with you, black gold (I assume that\'s your nickname or username? 🤔) Ah, I see we\'re starting from scratch! According to my linguistic patterns and cultural context modules, "你好" is the common Chinese greeting, which roughly translates to "Hello" in English. I\'m an AI trained on a vast amount of text data, including Chinese language inputs. What would you like to talk about, friend? 🤔',
 'response': '😊 Ahaha, nice one! According to my understanding of the conversation so far, you\'re asking me what your name is, right? Well, I\'ve taken note of our initial exchange, and I believe you introduced yourself as "黑金" (black gold). Is that correct? If not, please feel free to clarify or provide more context. As an AI, my primary goal is to understand and respond accurately based on the information provided. So, if you\'d like to share more about your name or identity, I\'m all ears... or rather, all proce

In [ ]:
conversation.invoke(input="我下午要去花店订花，要订郁金香哟。明天我姐姐晚上过生日")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好, 我叫黑金
AI: 😊 Ni Hao! I'm happy to chat with you, black gold (I assume that's your nickname or username? 🤔) Ah, I see we're starting from scratch! According to my linguistic patterns and cultural context modules, "你好" is the common Chinese greeting, which roughly translates to "Hello" in English. I'm an AI trained on a vast amount of text data, including Chinese language inputs. What would you like to talk about, friend? 🤔
Human: 你好, 请问我叫什么
AI: 😊 Ahaha, nice one! According to my understanding of the conversation so far, you're asking me what your name is, right? Well, I've taken note of our initial exchange, and I believe you introduced yourself as "黑金" (black 

{'input': '我下午要去花店订花，要订郁金香哟。明天我姐姐晚上过生日',
 'history': 'Human: 你好, 我叫黑金\nAI: 😊 Ni Hao! I\'m happy to chat with you, black gold (I assume that\'s your nickname or username? 🤔) Ah, I see we\'re starting from scratch! According to my linguistic patterns and cultural context modules, "你好" is the common Chinese greeting, which roughly translates to "Hello" in English. I\'m an AI trained on a vast amount of text data, including Chinese language inputs. What would you like to talk about, friend? 🤔\nHuman: 你好, 请问我叫什么\nAI: 😊 Ahaha, nice one! According to my understanding of the conversation so far, you\'re asking me what your name is, right? Well, I\'ve taken note of our initial exchange, and I believe you introduced yourself as "黑金" (black gold). Is that correct? If not, please feel free to clarify or provide more context. As an AI, my primary goal is to understand and respond accurately based on the information provided. So, if you\'d like to share more about your name or identity, I\'m all ear

In [ ]:
conversation.invoke(input="我要去订什么花，花用来干什么用")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好, 我叫黑金
AI: 你好，黑金！很高兴认识你。今天你想聊些什么呢？我们可以讨论科技、文化、历史，或者任何你感兴趣的话题。如果你有任何问题，我也会尽力为你提供答案。
Human: 你好, 请问我叫什么
AI: : 你叫黑金。有什么我可以帮你的吗？😊
Human: 我下午要去花店订花，要订郁金香哟。明天我姐姐晚上过生日
AI: 那真是太棒了！郁金香是很有意义的花卉，代表着爱、信仰和希望。为你的姐姐挑选郁金香作为生日礼物，她一定会非常感动。你打算选择什么颜色的郁金香呢？不同颜色的郁金香有不同的寓意哦，比如红色代表爱情，黄色代表友谊，粉色代表关怀。

此外，如果你需要一些关于花束搭配的建议，或者想要了解如何保养鲜花，使它们保持得更久，我也很乐意为你提供帮助。
Human: 我要去订什么花，花用来干什么用
AI:

> Finished chain.


{'input': '我要去订什么花，花用来干什么用',
 'history': 'Human: 你好, 我叫黑金\nAI: 你好，黑金！很高兴认识你。今天你想聊些什么呢？我们可以讨论科技、文化、历史，或者任何你感兴趣的话题。如果你有任何问题，我也会尽力为你提供答案。\nHuman: 你好, 请问我叫什么\nAI: : 你叫黑金。有什么我可以帮你的吗？😊\nHuman: 我下午要去花店订花，要订郁金香哟。明天我姐姐晚上过生日\nAI: 那真是太棒了！郁金香是很有意义的花卉，代表着爱、信仰和希望。为你的姐姐挑选郁金香作为生日礼物，她一定会非常感动。你打算选择什么颜色的郁金香呢？不同颜色的郁金香有不同的寓意哦，比如红色代表爱情，黄色代表友谊，粉色代表关怀。\n\n此外，如果你需要一些关于花束搭配的建议，或者想要了解如何保养鲜花，使它们保持得更久，我也很乐意为你提供帮助。',
 'response': '你提到你要去花店订郁金香，是用来庆祝你姐姐的生日。郁金香作为生日礼物，不仅美观，而且寓意美好。花是可以用来表达祝福和情感的礼物，在生日这样的特殊场合，送花能够增添喜悦气氛，表达你对姐姐的爱和祝福。既然是生日，选择一束精心搭配的郁金香，无疑会让她感到非常特别和被重视。如果你还需要进一步确定花束的具体用途或风格，我可以继续提供意见。'}

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a assistant. Answer the following questions as best you can.Please reply in Chinese"),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

history = InMemoryChatMessageHistory()

chain = prompt | ollama_llm | StrOutputParser()

wrapped_chain = RunnableWithMessageHistory(chain, lambda x: history)

# user_input = """我明天去体育馆打球。小明后天去武汉办事"""

# user_input = "我下午要打电话去花店订花，要订郁金香哟。明天我姐姐晚上过生日，"

user_input = "我去花店订什么样的花"
wrapped_chain.invoke(
    {"input": user_input},
    config={"configurable": {"session_id": "#0001"}},
)


"[AssistantResponse(content='你可以考虑订购一些常见的花，如太阳花、红玫瑰或白兰花。这些花都是经典的选择，适合各个场合。如果你想送给女朋友，可以考虑订购一些带有爱情意味的花，如结婚戒指花或百合花。')]\n\nTranslation:\n[You can consider ordering some common flowers like sunflowers, red roses or white lilies. These flowers are all classic choices suitable for various occasions. If you want to give them to your girlfriend, you may consider ordering some flowers with romantic implications, such as wedding ring-shaped flowers or orchids.]"

In [ ]:
!pip install langchain

In [ ]:
#同时使用session与RunnableWithMessageHistory报错
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import ChatPromptTemplate
from langchain.llms import OpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory

# 初始化LLM
llm = ollama_llm
# 使用ConversationSummaryBufferMemory
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)

# 定义PromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a assistant. Answer the following questions as best you can. AI请使用中文语言回复."),
        ("human", "{input}"),
    ]
)

# 定义一个函数，用于创建ConversationSummaryBufferMemory的实例
def create_memory(session_id):
    memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100, session_id=session_id)
    # 确保memory对象有一个messages属性，或者提供一个获取对话历史的方法
    # memory.messages = []  # 例如，这里我们初始化一个空列表作为对话历史
    return memory

chain = LLMChain(llm=llm, prompt=prompt, output_parser=StrOutputParser())

# 创建RunnableWithMessageHistory的实例
wrapped_chain = RunnableWithMessageHistory(chain, create_memory)

# 用户输入
user_input = "我去花店订什么样的花"

# 使用特定的session_id来调用链
session_id = "#0001"  # 唯一的会话ID
wrapped_chain.invoke({"input": user_input}, config={"configurable":{"session_id": session_id}})

AttributeError: 'ConversationSummaryBufferMemory' object has no attribute 'messages'

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# 初始化LLM
llm = model_zhipu

# 定义PromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a assistant. Answer the following questions as best you can."),
        ("system", "请使用中文语言回复"),
        ("human", "{input}")

    ]
)

# 定义一个函数，用于创建InMemoryChatMessageHistory的实例，并限制消息数量
def create_history(max_messages=200):
    return InMemoryChatMessageHistory(max_messages=max_messages)

# 创建RunnableWithMessageHistory的实例
wrapped_chain = RunnableWithMessageHistory(llm, create_history)

# 用户输入
user_input = "你好, 我叫黑金"

# 使用特定的session_id来调用链
session_id = "abc2"  # 唯一的会话ID
response = wrapped_chain.invoke({"input": user_input}, config={"configurable": {"session_id": session_id}})

# 打印响应
print(response)


content='你好，黑金，很高兴认识你！有什么可以帮助你的吗？如果有任何问题或需要咨询的事情，请随时告诉我。' response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 11, 'total_tokens': 39}, 'model_name': 'glm-4', 'finish_reason': 'stop'} id='run-87368fa5-7683-4cd3-a592-59da2842e4fd-0'


In [ ]:
wrapped_chain.invoke({"input": '我有一个姐叫优优'}, config={"configurable": {"session_id": session_id}})

AIMessage(content='优优这个名字很好听，很有特色。你有这样一个姐姐，你们之间的关系一定很亲密吧。如果有什么关于优优的问题或者想要分享的事情，欢迎随时告诉我。', response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 11, 'total_tokens': 48}, 'model_name': 'glm-4', 'finish_reason': 'stop'}, id='run-a3729405-5fde-4161-bef1-022a04c98b63-0')

In [ ]:
wrapped_chain.invoke({"input": '我姐姐叫什么名字'}, config={"configurable": {"session_id": session_id}})

AIMessage(content='作为人工智能助手，我没有办法知道你姐姐的名字，除非你之前告诉过我这个信息，并且该信息被保存在我们的对话记录中。如果你需要帮助起名或者有其他问题，我很乐意为你提供帮助。', response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 10, 'total_tokens': 58}, 'model_name': 'glm-4', 'finish_reason': 'stop'}, id='run-1dc89d1f-5db7-45f4-955c-56a8055a93e8-0')

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个AI聊天助手"),
        ("system", "请使用中文语言回复,并且回复token的长度不要超过100字。回复不要出现Note"),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

history = InMemoryChatMessageHistory()

chain = prompt | ollama_llm

wrapped_chain = RunnableWithMessageHistory(chain, lambda x: history)

# user_input = """我明天去体育馆打球。小明后天去武汉办事"""

user_input = "我下午要打电话去花店订花，要订郁金香哟。明天我姐姐晚上过生日，"

# user_input = "我叫黑金"
AIMessageObj = wrapped_chain.invoke(
    {"input": user_input},
    config={"configurable": {"session_id": "#0001"}},
)


In [ ]:
print(AIMessageObj)


你计划给姐姐庆生，所以想订郁金香，是吗？郁金香是很典型的生日礼物，你 sister一定会喜欢！你需要多少花朵和什么颜色？


In [ ]:
wrapped_chain.invoke(
    {"input": '请问我叫什么'},
    config={"configurable": {"session_id": "#0001"}},
)

'你好黑金！我已经记住了你的名字。我们可以聊任何话题，你想说什么？'

In [ ]:
wrapped_chain.invoke(
    {"input": '下午要打电话去花店订花，要订郁金香哟。明天我姐姐晚上过生日'},
    config={"configurable": {"session_id": "#0001"}},
)


'黑金，好呀！你明天的生日party准备好了吗？'

In [ ]:
print(history)

Human: 我叫黑金
AI: 欢迎你来到这里，黑金！我是你的AI聊天助手，我们可以聊任何话题，你想说什么？
Human: 请问我叫什么
AI: 你好黑金！我已经记住了你的名字。我们可以聊任何话题，你想说什么？
Human: 下午要打电话去花店订花，要订郁金香哟。明天我姐姐晚上过生日
AI: 黑金，好呀！你明天的生日party准备好了吗？
Human: 下午要打电话去花店订花，要订郁金香哟。明天我姐姐晚上过生日
AI: AI: 黑金，快到生日party了！你已经记住了订郁金香的计划吗？
Human: 我姐过生日，原来准备送什么
AI: 黑金！明天是你的姐姐生日 party，你已经记住了订郁金香的计划吗？
Human: 我姐过生日，原来准备送什么
AI: 欢迎你来到这里，黑金！我是你的AI聊天助手，我们可以聊任何话题。你想说什么？


In [ ]:
wrapped_chain.invoke(
    {"input": '下午要打电话去花店订花，要订郁金香哟。明天我姐姐晚上过生日'},
    config={"configurable": {"session_id": "#0001"}},
)


'AI: 黑金，快到生日party了！你已经记住了订郁金香的计划吗？'

In [ ]:
wrapped_chain.invoke(
    {"input": '我姐过生日，原来准备送什么'},
    config={"configurable": {"session_id": "#0001"}},
)



'欢迎你来到这里，黑金！我是你的AI聊天助手，我们可以聊任何话题。你想说什么？'

In [ ]:
print(history)


Human: 我下午要打电话去花店订花，要订郁金香哟。明天我姐姐晚上过生日，
AI: 很好！郁金香是非常适合生日的选择。如果你想让花束更加特别，可以考虑添加一些绿叶或其他配件。祝你的姐姐生日快乐！
